# Design Decisions with Python Functions



In [ ]:
using Base.Test

# Two primary reasons for defining functions:
1. Code reuse: 
    * Write and debug code once. Then I can use this same (correct) code many times
    * This makes upkeep/modifications simpler. When I think of an improved way of implementing something I only need to change it in one location.

2. Procedural Decomposition:
    * A function should do one thing, not multiple things.
    * This can become a matter of style
    

## Function Style According to Mark Thomason
![Mark Thomason](./mark_thomason.jpg)

## The entire function should be visible on your screen within your editor.
## If your function doesn't fit on your screen, get a bigger screen

### What are the implications of these heuristic?
#### Function size changes with age?

## Exercise: Define a function to get a positive integer from a user.
### Requirements
1. Use an infinite while loop
1. Use the input function
1. Keep prompting the user for input until a valid positive integer is provided

We'll use a try/except block to account for users entering non-integer value

```Julia
try
    # Get input from user
    # convert to an integer (this where we could get an exception
    # test for positivity
catch error
    
    # if we get an input that we can't convert to an integer, we need to do something
end
```

In [ ]:
function input(;prompt="Enter")
    println(prompt)
    return readline()
end

In [ ]:
function get_pos_integer(;prompt="Enter a positive integer")
    while true
        num = input(prompt)
        try
            num = parse(Int64, num)
            if num > 0
                return num
            end
        catch error
            if isa(error, ArgumentError)
                continue
            end
        end
    end
end


## Does this function do one thing?
## Could we break it into smaller pieces?

### Write a function to test whether a number is positive

In [ ]:
function ispositive(x)
    return x > 0
end

In [ ]:
@test ispositive(5)
@test !ispositive(-1)
@test !ispositive(0)

In [ ]:
function getint(sint)
    return parse(Int64, sint)
end


In [ ]:
parse(Int64, "4.7")

In [ ]:
@test getint("543")==543
@test_throws ArgumentError getint("4.7")


### Using `getint` and `ispositive` rewrite `get_pos_integer`

In [ ]:
function get_pos_integer2(prompt="Enter a positive integer")
    while true
        num = input(prompt)
        try
            num = getint(num)
            if ispositive(num)
                return num
            end
        catch error
            if isa(error, ArgumentError)
                continue
            end
        end
    end
end


In [ ]:
# Enter '5'
@test get_pos_integer2()==5

# Enter '7'
@test get_pos_integer2()==7


## Exercise

Following the same style as `get_pos_integer`, write a function `get_value`. `get_value` takes as arguments:

1. A positional argument `converter` that is a function that takes as input a string and returns the desired value
1. A positional argument `tester` that is a function that takes as input a value and returns `True` or `False` depending on whether a desired condition is satisfied.
1. A keyword argument `prompt` that is the prompt to use with `input`.

Test the function with `getint` and `ispositive`.

In [ ]:
### BEGIN SOLUTION

function get_value(converter, tester; prompt="enter a positive integer: ")
    while true
        num = input(prompt)
        try
            num = converter(num)
            if tester(num)
                return num
            end
        catch error
            if isa(error, ArgumentError)
                continue
            end
        end
    end
end
### END SOLUTION

In [ ]:
get_value(getint, ispositive)

## Exercise

Modify `get_three_words` to take a string `entry` splits it on white spaces and returns a list of three words. If the list is not three words long, raise a `ValueError`.

In [ ]:
function get_three_words(entry)
    ### BEGIN SOLUTION
    entry = split(entry)
    if length(entry) != 3
        error("Error: Must enter three words")
    end
    return entry
end
    ### END SOLUTION

In [ ]:
get_three_words("Brian Earl Chapman")

In [ ]:
print(get_three_words("Brian Chapman"))
print("I got here")

In [ ]:
@test length(get_three_words("Brian Earl Chapman")) == 3;

## Exercise

Modify `test_ascending` to take in a sequence and test if the elements in `values` are in ascending order.

**Hint:** Use the `all` function.

In [ ]:
?all

In [ ]:
function test_ascending(values)
    ### BEGIN SOLUTION
    return all([values[i] < values[i+1] for i in range(1,length(values)-1)])
    ### END SOLUTION
end

In [ ]:
?range

In [ ]:
test_ascending(get_three_words("1 2 3"))

In [ ]:
@test test_ascending(("Argos", "Helios", "Zeus"))
@test !test_ascending(("Argos", "Zeus", "Helios"))
@test !test_ascending(("argos", "Helios", "Zeus"));


In [ ]:
get_value(get_three_words, 
          test_ascending, 
          prompt="enter three words in ascending alphabetical order separated by spaces: ")


In [ ]:
function foo(a=[])
    return push!(a,5)
end
println(foo())
println(foo())

In [ ]:
function i_hate_foo(a=nothing)
    if a == nothing
        a = []
    end
    return push!(a, 5)
end
println(i_hate_foo())
println(i_hate_foo())

## Exercise

Write a function ``sumthings`` that takes a variable number of arguments and returns the sum of their values.

#### Challenge: Can you do this with a single Python statement within `sumthings`?

In [ ]:
function sumthings(x...)
    ### BEGIN SOLUTION
    return sum(x)
end
    ### END SOLUTION

In [ ]:
@test sumthings(1,2,3)==6
@test sumthings(1)==1
@test_throws StackOverflowError sumthings();


### Variable number of keyword arguments

A function definition that looks like this

```Julia

function some_function(;kwargs...):
    ### BLOCK OF CODE
end
```

has a variable number of keyword arguments

The variable number of positional arguments are passed to the function as a **list.**

In [ ]:
function demo2(;kwargs...)
    for k in kwargs
        k,v = k
        println(k," ", v)
    end
end
demo2(print="No way", age=29, favorite_number=8, luck="bad")

## Exercise

Write a function `keep_numeric` that takes a variable number of keyword arguments and returns a dictionary consisting of the kwargs passed to `keep_numeric` where the value is an integer

**Challenge**: Return a dictionary of kwargs where the value is any numeric value

In [ ]:
isa(5.+0im,Number)

In [ ]:
### BEGIN SOLUTION

function keep_numeric(;kwargs...)
    newd = Dict()
    for k in kwargs
        println(k)
        if isa(k[2], Number)
            newd[k[1]] = k[2]
        end
    end
    return newd
end

In [ ]:
keep_numeric(name="Brian", age=29, race="Caucasian", weight=150)

In [ ]:
Dict(i=>i for i in range(1,5) if i%2==0)

In [ ]:
function keep_numeric(;kwargs...)
    return Dict(k[1]=>k[2] for k in kwargs if isa(k[2],Number))
end


In [ ]:
@test keep_numeric(name="Brian", age=29, 
            race="Caucasian", weight=150) ==
    Dict(:age=> 29, :weight=>150)

In [ ]:
@test keep_numeric(name="Brian", age=29.5, 
    race="Caucasian", 
    weight=150.4) == 
Dict(:age=> 29.5, :weight => 150.4)

In [ ]:
@test keep_numeric(name="Brian", age=29.5, 
        race="Caucasian", weight=150.4) ==
    Dict(:age=> 29.5, :weight=> 150.4)